# TensorFlow Pokemon Course

In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing


## Read data

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TF-Pkm-Course/pokemon.csv')

In [4]:
df.columns

Index(['Number', 'Name', 'Type_1', 'Type_2', 'Total', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'isLegendary',
       'Color', 'hasGender', 'Pr_Male', 'Egg_Group_1', 'Egg_Group_2',
       'hasMegaEvolution', 'Height_m', 'Weight_kg', 'Catch_Rate',
       'Body_Style'],
      dtype='object')

## Preprocess

In [7]:
df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]


<bound method NDFrame.head of      isLegendary  Generation   Type_1  ... Height_m  Weight_kg       Body_Style
0          False           1    Grass  ...     0.71        6.9        quadruped
1          False           1    Grass  ...     0.99       13.0        quadruped
2          False           1    Grass  ...     2.01      100.0        quadruped
3          False           1     Fire  ...     0.61        8.5   bipedal_tailed
4          False           1     Fire  ...     1.09       19.0   bipedal_tailed
..           ...         ...      ...  ...      ...        ...              ...
716         True           6     Dark  ...     5.79      203.0        two_wings
717         True           6   Dragon  ...     5.00      305.0  serpentine_body
718         True           6     Rock  ...     0.71        8.8        head_arms
719         True           6  Psychic  ...     0.51        9.0        head_only
720         True           6     Fire  ...     1.70      195.0        quadruped

[721 rows

In [8]:
display(df.head(10))

,isLegendary,Generation,Type_1,Type_2,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Color,Egg_Group_1,Height_m,Weight_kg,Body_Style
0,False,1,Grass,Poison,45,49,49,65,65,45,Green,Monster,0.71,6.9,quadruped
1,False,1,Grass,Poison,60,62,63,80,80,60,Green,Monster,0.99,13.0,quadruped
2,False,1,Grass,Poison,80,82,83,100,100,80,Green,Monster,2.01,100.0,quadruped
3,False,1,Fire,NaN,39,52,43,60,50,65,Red,Monster,0.61,8.5,bipedal_tailed
4,False,1,Fire,NaN,58,64,58,80,65,80,Red,Monster,1.09,19.0,bipedal_tailed
5,False,1,Fire,Flying,78,84,78,109,85,100,Red,Monster,1.70,90.5,bipedal_tailed
6,False,1,Water,NaN,44,48,65,50,64,43,Blue,Monster,0.51,9.0,bipedal_tailed
7,False,1,Water,NaN,59,63,80,65,80,58,Blue,Monster,0.99,22.5,bipedal_tailed
8,False,1,Water,NaN,79,83,100,85,105,78,Blue,Monster,1.60,85.5,bipedal_tailed
9,False,1,Bug,NaN,45,30,35,20,20,45,Green,Bug,0.30,2.9,insectoid


In [10]:
#create dummy variables for one's with many values
def dummy_creation(df, dummy_categories):
    for i in dummy_categories:
        df_dummy = pd.get_dummies(df[i])
        df = pd.concat([df,df_dummy],axis=1)
        df = df.drop(i, axis=1)
    return(df)


In [11]:
df = dummy_creation(df, ['Egg_Group_1', 'Body_Style', 'Color','Type_1', 'Type_2'])


## Split and Normalize Data


In [12]:
def train_test_splitter(DataFrame, column):
    df_train = DataFrame.loc[df[column] != 1]
    df_test = DataFrame.loc[df[column] == 1]

    df_train = df_train.drop(column, axis=1)
    df_test = df_test.drop(column, axis=1)

    return(df_train, df_test)

df_train, df_test = train_test_splitter(df, 'Generation')


In [13]:
def label_delineator(df_train, df_test, label):
    
    train_data = df_train.drop(label, axis=1).values
    train_labels = df_train[label].values
    test_data = df_test.drop(label,axis=1).values
    test_labels = df_test[label].values
    return(train_data, train_labels, test_data, test_labels)


In [14]:
train_data, train_labels, test_data, test_labels = label_delineator(df_train, df_test, 'isLegendary')

In [15]:
def data_normalizer(train_data, test_data):
    train_data = preprocessing.MinMaxScaler().fit_transform(train_data)
    test_data = preprocessing.MinMaxScaler().fit_transform(test_data)
    return(train_data, test_data)

train_data, test_data = data_normalizer(train_data, test_data)


## Model

In [16]:
length = train_data.shape[1]

model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='relu', input_shape=[length,]))
model.add(keras.layers.Dense(2, activation='softmax'))

In [17]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(train_data, train_labels, epochs=400)

Epoch 1/400
18/18 [==============================] - 1s 2ms/step - loss: 0.6692 - accuracy: 0.5789
Epoch 2/400
18/18 [==============================] - 0s 1ms/step - loss: 0.4830 - accuracy: 0.9246
Epoch 3/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3926 - accuracy: 0.9263
Epoch 4/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3443 - accuracy: 0.9263
Epoch 5/400
18/18 [==============================] - 0s 1ms/step - loss: 0.3158 - accuracy: 0.9263
Epoch 6/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2974 - accuracy: 0.9263
Epoch 7/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2847 - accuracy: 0.9263
Epoch 8/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2752 - accuracy: 0.9263
Epoch 9/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2676 - accuracy: 0.9263
Epoch 10/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2613 - accuracy: 0.9263
Epoch 11/

In [20]:
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print(f'Our test accuracy was {accuracy_value}')

5/5 [==============================] - 0s 3ms/step - loss: 0.0392 - accuracy: 0.9801
Our test accuracy was 0.9801324605941772


We can also use our model to predict specific Pokémon, or at least have it tell us which status the Pokémon is most likely to have, with model.predict. All it needs to predict a Pokémon is the data for that Pokémon itself. We're providing that by selecting a certain index of test_data:

In [21]:
def predictor(test_data, test_labels, index):
    prediction = model.predict(test_data)
    if np.argmax(prediction[index]) == test_labels[index]:
        print(f'This was correctly predicted to be a \"{test_labels[index]}\"!')
    else:
        print(f'This was incorrectly predicted to be a \"{np.argmax(prediction[index])}\". It was actually a \"{test_labels[index]}\".')
        return(prediction)


Let's look at Mewtwo, one of the more well-known legendary Pokemon.

In [22]:
predictor(test_data, test_labels, 149)

This was correctly predicted to be a "True"!
